In [1]:
%load_ext rqalpha

In [2]:
%%rqalpha -fq 1m -sp 0.02 -s 20180701 -e 20190930 -o result.pkl -p -bm 000001.XSHG --account stock 200000 --plot

# 可以自己import我们平台支持的第三方python模块，比如pandas、numpy等。
import talib
import pandas as pd

import numpy as np
import functools
import math
import datetime

def convert_int_to_datetime(dt_int):
    dt_int = int(dt_int)
    year, r = divmod(dt_int, 10000000000)
    month, r = divmod(r, 100000000)
    day, r = divmod(r, 1000000)
    hour, r = divmod(r, 10000)
    minute, second = divmod(r, 100)
    return datetime.datetime(year, month, day, hour, minute, second)


def KDJ_tl(df) :
    #计算KDJ指标
    high=np.array(df['high'])
    low=np.array(df['low'])
    close=np.array(df['close'])
    df['KDJ_k'],df['KDJ_d']=talib.STOCH(high,low,close,fastk_period=9,slowk_period=3,slowk_matype=0,slowd_period=3,slowd_matype=0)
    df['KDJ_j']=df['KDJ_k']*3-df['KDJ_d']*2
    KDJ_k=df['KDJ_k']
    KDJ_d=df['KDJ_d']
    KDJ_j=df['KDJ_j']

    return KDJ_k,KDJ_d,KDJ_j

def MACD_tl(df) :
    ###计算MACD
    close = [float(x) for x in df['close']]
    highPrice = [float(x) for x in df['high']]
    lowPrice = [float(x) for x in df['low']]
    df['macdDIFF'],df['macdDEA'],df['macd'] =talib.MACDEXT(np.array(close), fastperiod=12,fastmatype=1,slowperiod=26, slowmatype=1, signalperiod=9, signalmatype=1)
    df['macd'] = df['macd'] * 2
    macdDIFF=df['macdDIFF']
    macdDEA=df['macdDEA']
    macd=df['macd']
    return macdDIFF,macdDEA,macd
#选股
def select_stock(context):
    
    stocks= []

    # 获取股票的列表
    df_stocks=all_instruments('CS')
    i=0
    for stock in df_stocks['order_book_id'].values :
        stock_name=df_stocks['symbol'][i]
        bar=history_bars(stock, 200, '1d', fields=None, skip_suspended=True, include_now=False)
        df = pd.DataFrame.from_dict(bar)
        if 'datetime' in df.columns :
            df['datetime'] = df['datetime'].map(lambda x: convert_int_to_datetime(x))
            df.set_index('datetime', inplace=True)
            df.index.name = ''
        if len(df)>=80 :
            #计算日线数据
            #计算均价
            df['MA5'] = df['close'].rolling(5).mean()
            df['MA10'] = df['close'].rolling(10).mean()
            
            #计算均量
            df['MA5vol'] = df['volume'].rolling(5).mean()
            df['MA10vol'] = df['volume'].rolling(10).mean()
            #计算KDJ
            KDJ_k,KDJ_d,KDJ_j=KDJ_tl(df)
            #计算MACD
            macdDIFF,macdDEA,macd=MACD_tl(df)
            
            if df['MA5'][-1]>df['MA10'][-1] and df['MA5'][-2]<=df['MA10'][-2] and \
            df['MA5vol'][-1]>df['MA10vol'][-1] and df['MA5vol'][-2]<=df['MA10vol'][-2] and \
            macdDIFF[-1]>macdDEA[-1] and macdDIFF[-2]<=macdDEA[-2] and macd[-1]>0 :
                print("股票:",stock,stock_name,"在:",df.index[-1].date(),"收盘后被选入股池")     
                stocks.append([df_stocks['order_book_id'][i],df_stocks['symbol'][i],df.index[-1].date()])                               
                                
        i += 1
    return stocks

#买入
def buy_stock(context):
    pass

#卖出
def sell_stock(context):
    pass

#加仓已持仓股
def add_hold(context) :
    pass
    
def update_select_record(context):
    pass
    
def update_buy_record(context):
    pass
    
def update_sell_record(context):
    pass   


# 在这个方法中编写任何的初始化逻辑。context对象将会在你的算法策略的任何方法之间做传递。
def init(context):
    # 在context中保存全局变量
    #context.s1 = "000001.XSHE"   
    # 实时打印日志
    logger.info("RunInfo: {}".format(context.run_info))
    

# before_trading此函数会在每天策略交易开始前被调用，当天只会被调用一次
def before_trading(context):    
    #买入选股
    #logger.info("选股依据的时间:",context.now,"前一交易日收盘")
    select_stock(context)
    

# 你选择的证券的数据更新将会触发此段逻辑，例如日或分钟历史数据切片或者是实时数据切片更新
def handle_bar(context, bar_dict):
    hour=context.now.hour
    minute=context.now.minute
    #卖出
    #sell_stock(context)
    #买入
    if hour==9 and minute==31 :
        buy_stock(context)
    # 开始编写你的主要的算法逻辑

    # bar_dict[order_book_id] 可以拿到某个证券的bar信息
    # context.portfolio 可以拿到现在的投资组合信息

    # 使用order_shares(id_or_ins, amount)方法进行落单

    # TODO: 开始编写你的算法吧！
    #order_shares(context.s1, 1000)

# after_trading函数会在每天交易结束后被调用，当天只会被调用一次
def after_trading(context):
    pass


Aborted!
